### Import Required Libraries and Set Up Environment Variables

In [1]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json
import os
from datetime import datetime
## Load the NASA_API_KEY from the env file
load_dotenv()
NASA_API_KEY = os.getenv('NASA_API_KEY')

### CME Data

In [2]:
# Set the base URL to NASA's DONKI API:
base_url = "https://api.nasa.gov/DONKI/"

# Set the specifier for CMEs:
CME = "CME"

# Search for CMEs published between a begin and end date
startDate = "2013-05-01"
endDate   = "2024-05-01"

# Build URL for CME
cme_url = f"{base_url}{CME}?startDate={startDate}&endDate={endDate}&api_key={NASA_API_KEY}"
print(cme_url)

https://api.nasa.gov/DONKI/CME?startDate=2013-05-01&endDate=2024-05-01&api_key=erI3TXXxMWNxoAuxWTN120s4vc27cAPLtvgkX2iF


In [3]:
# Make a "GET" request for the CME URL and store it in a variable named cme_response
cme_response = requests.get(cme_url)
if cme_response.status_code == 200:
    print("CME Request Successful")
else:
    print("The CME request failed: {cme_response.status_code}!")

CME Request Successful


In [4]:
# Convert the response variable to json and store it as a variable named cme_json
cme_json = cme_response.json()

In [5]:
# Preview the first result in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(cme_json[0], indent=4))

{
    "activityID": "2013-05-01T03:12:00-CME-001",
    "catalog": "M2M_CATALOG",
    "startTime": "2013-05-01T03:12Z",
    "instruments": [
        {
            "displayName": "SOHO: LASCO/C2"
        },
        {
            "displayName": "SOHO: LASCO/C3"
        },
        {
            "displayName": "STEREO A: SECCHI/COR2"
        },
        {
            "displayName": "STEREO B: SECCHI/COR2"
        }
    ],
    "sourceLocation": "",
    "activeRegionNum": null,
    "note": "",
    "submissionTime": "2013-08-07T16:54Z",
    "versionId": 1,
    "link": "https://webtools.ccmc.gsfc.nasa.gov/DONKI/view/CME/2349/-1",
    "cmeAnalyses": [
        {
            "isMostAccurate": true,
            "time21_5": "2013-05-01T07:07Z",
            "latitude": 12.0,
            "longitude": -120.0,
            "halfAngle": 36.0,
            "speed": 860.0,
            "type": "C",
            "featureCode": "null",
            "imageType": null,
            "measurementTechnique": "null",
   

In [6]:
# Convert cme_json to a Pandas DataFrame 
cme_df = pd.DataFrame(cme_json)
# Keep only the columns: activityID, startTime, linkedEvents
cme_df = cme_df[['activityID', 'startTime', 'linkedEvents']]
cme_df.head()

,activityID,startTime,linkedEvents
0,2013-05-01T03:12:00-CME-001,2013-05-01T03:12Z,[{'activityID': '2013-05-04T04:52:00-IPS-001'}]
1,2013-05-02T05:24:00-CME-001,2013-05-02T05:24Z,None
2,2013-05-02T14:36:00-CME-001,2013-05-02T14:36Z,None
3,2013-05-03T18:00:00-CME-001,2013-05-03T18:00Z,None
4,2013-05-03T22:36:00-CME-001,2013-05-03T22:36Z,[{'activityID': '2013-05-07T04:37:00-IPS-001'}]


In [7]:
# Notice that the linkedEvents column allows us to identify the corresponding GST
# Remove rows with missing 'linkedEvents' since we won't be able to assign these to GSTs
cme_df = cme_df[cme_df['linkedEvents'].notna()]
cme_df.reset_index(drop=True, inplace=True)
cme_df.head()

,activityID,startTime,linkedEvents
0,2013-05-01T03:12:00-CME-001,2013-05-01T03:12Z,[{'activityID': '2013-05-04T04:52:00-IPS-001'}]
1,2013-05-03T22:36:00-CME-001,2013-05-03T22:36Z,[{'activityID': '2013-05-07T04:37:00-IPS-001'}]
2,2013-05-09T19:29:00-CME-001,2013-05-09T19:29Z,[{'activityID': '2013-05-12T23:30:00-IPS-001'}]
3,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,[{'activityID': '2013-05-13T01:53:00-FLR-001'}...
4,2013-05-13T16:18:00-CME-001,2013-05-13T16:18Z,[{'activityID': '2013-05-13T15:40:00-FLR-001'}...


In [8]:
# Notice that the linkedEvents sometimes contains multiple events per row
# Write a nested for loop that iterates first over each row in the cme DataFrame (using the index)
# and then iterates over the values in 'linkedEvents' 
# and adds the elements individually to a list of dictionaries where each row is one element 

# Initialize an empty list to store the expanded rows
expanded_rows = []

# Iterate over each index in the DataFrame
for i in cme_df.index:
    activity_ID = cme_df.loc[i, 'activityID']
    start_Time = cme_df.loc[i, 'startTime']
    linked_Events = cme_df.loc[i, 'linkedEvents']
    # Iterate over each dictionary in the list
    for event in linked_Events:
        # Append a new dictionary to the expanded_rows list for each dictionary item and corresponding 'activityID' and 'startTime' value
      expanded_rows.append({
         'activityID': activity_ID,
         'startTime': start_Time,
         'linkedEvents': event
      })
# Create a new DataFrame from the expanded rows
expanded_df = pd.DataFrame(expanded_rows)
expanded_df.head()

,activityID,startTime,linkedEvents
0,2013-05-01T03:12:00-CME-001,2013-05-01T03:12Z,{'activityID': '2013-05-04T04:52:00-IPS-001'}
1,2013-05-03T22:36:00-CME-001,2013-05-03T22:36Z,{'activityID': '2013-05-07T04:37:00-IPS-001'}
2,2013-05-09T19:29:00-CME-001,2013-05-09T19:29Z,{'activityID': '2013-05-12T23:30:00-IPS-001'}
3,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,{'activityID': '2013-05-13T01:53:00-FLR-001'}
4,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,{'activityID': '2013-05-13T04:12:00-SEP-001'}


In [9]:
#print(expanded_df.columns.tolist())

In [10]:
# Create a function called extract_activityID_from_dict that takes a dict as input such as in linkedEvents
# and verify below that it works as expected using one row from linkedEvents as an example
# Be sure to use a try and except block to handle errors
def extract_activityID_from_dict(input_dict):
        try:
                return input_dict['activityID']
        # Log the error or print it for debugging
        except (ValueError, TypeError, KeyError) as e:
                return None

test_event = expanded_df.loc[0, 'linkedEvents']
print("Test Event: ", test_event)
extracted_activityID = extract_activityID_from_dict(test_event)
print("Extracted ActivityID: ", extracted_activityID)

Test Event:  {'activityID': '2013-05-04T04:52:00-IPS-001'}
Extracted ActivityID:  2013-05-04T04:52:00-IPS-001


In [11]:
# Apply this function to each row in the 'linkedEvents' column (you can use apply() and a lambda function)
# and create a new column called 'GST_ActivityID' using loc indexer:
expanded_df['GST_ActivityID'] = expanded_df['linkedEvents'].apply(lambda x: extract_activityID_from_dict(x))
expanded_df = expanded_df[expanded_df['GST_ActivityID'].notna()]
#missing_values = expanded_df['GST_ActivityID'].isna().sum()
#missing_values
expanded_df

,activityID,startTime,linkedEvents,GST_ActivityID
0,2013-05-01T03:12:00-CME-001,2013-05-01T03:12Z,{'activityID': '2013-05-04T04:52:00-IPS-001'},2013-05-04T04:52:00-IPS-001
1,2013-05-03T22:36:00-CME-001,2013-05-03T22:36Z,{'activityID': '2013-05-07T04:37:00-IPS-001'},2013-05-07T04:37:00-IPS-001
2,2013-05-09T19:29:00-CME-001,2013-05-09T19:29Z,{'activityID': '2013-05-12T23:30:00-IPS-001'},2013-05-12T23:30:00-IPS-001
3,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,{'activityID': '2013-05-13T01:53:00-FLR-001'},2013-05-13T01:53:00-FLR-001
4,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,{'activityID': '2013-05-13T04:12:00-SEP-001'},2013-05-13T04:12:00-SEP-001
...,...,...,...,...
1709,2024-04-24T01:48:00-CME-001,2024-04-24T01:48Z,{'activityID': '2024-04-26T00:17:00-IPS-001'},2024-04-26T00:17:00-IPS-001
1710,2024-04-24T15:05:00-CME-001,2024-04-24T15:05Z,{'activityID': '2024-04-24T14:02:00-FLR-001'},2024-04-24T14:02:00-FLR-001
1711,2024-04-25T18:24:00-CME-001,2024-04-25T18:24Z,{'activityID': '2024-04-25T17:03:00-FLR-001'},2024-04-25T17:03:00-FLR-001
1712,2024-04-30T00:36:00-CME-001,2024-04-30T00:36Z,{'activityID': '2024-04-30T00:46:00-FLR-001'},2024-04-30T00:46:00-FLR-001


In [12]:
# Remove rows with missing GST_ActivityID, since we can't assign them to GSTs:
expanded_df = expanded_df.explode('GST_ActivityID', ignore_index=True).dropna()

expanded_df

,activityID,startTime,linkedEvents,GST_ActivityID
0,2013-05-01T03:12:00-CME-001,2013-05-01T03:12Z,{'activityID': '2013-05-04T04:52:00-IPS-001'},2013-05-04T04:52:00-IPS-001
1,2013-05-03T22:36:00-CME-001,2013-05-03T22:36Z,{'activityID': '2013-05-07T04:37:00-IPS-001'},2013-05-07T04:37:00-IPS-001
2,2013-05-09T19:29:00-CME-001,2013-05-09T19:29Z,{'activityID': '2013-05-12T23:30:00-IPS-001'},2013-05-12T23:30:00-IPS-001
3,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,{'activityID': '2013-05-13T01:53:00-FLR-001'},2013-05-13T01:53:00-FLR-001
4,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,{'activityID': '2013-05-13T04:12:00-SEP-001'},2013-05-13T04:12:00-SEP-001
...,...,...,...,...
1709,2024-04-24T01:48:00-CME-001,2024-04-24T01:48Z,{'activityID': '2024-04-26T00:17:00-IPS-001'},2024-04-26T00:17:00-IPS-001
1710,2024-04-24T15:05:00-CME-001,2024-04-24T15:05Z,{'activityID': '2024-04-24T14:02:00-FLR-001'},2024-04-24T14:02:00-FLR-001
1711,2024-04-25T18:24:00-CME-001,2024-04-25T18:24Z,{'activityID': '2024-04-25T17:03:00-FLR-001'},2024-04-25T17:03:00-FLR-001
1712,2024-04-30T00:36:00-CME-001,2024-04-30T00:36Z,{'activityID': '2024-04-30T00:46:00-FLR-001'},2024-04-30T00:46:00-FLR-001


In [13]:
#unique_gst = expanded_df['GST_ActivityID'].unique()
#print(unique_gst)

In [14]:
# print out the datatype of each column in this DataFrame:
expanded_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1714 entries, 0 to 1713
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   activityID      1714 non-null   object
 1   startTime       1714 non-null   object
 2   linkedEvents    1714 non-null   object
 3   GST_ActivityID  1714 non-null   object
dtypes: object(4)
memory usage: 53.7+ KB


In [15]:
expanded_df

,activityID,startTime,linkedEvents,GST_ActivityID
0,2013-05-01T03:12:00-CME-001,2013-05-01T03:12Z,{'activityID': '2013-05-04T04:52:00-IPS-001'},2013-05-04T04:52:00-IPS-001
1,2013-05-03T22:36:00-CME-001,2013-05-03T22:36Z,{'activityID': '2013-05-07T04:37:00-IPS-001'},2013-05-07T04:37:00-IPS-001
2,2013-05-09T19:29:00-CME-001,2013-05-09T19:29Z,{'activityID': '2013-05-12T23:30:00-IPS-001'},2013-05-12T23:30:00-IPS-001
3,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,{'activityID': '2013-05-13T01:53:00-FLR-001'},2013-05-13T01:53:00-FLR-001
4,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,{'activityID': '2013-05-13T04:12:00-SEP-001'},2013-05-13T04:12:00-SEP-001
...,...,...,...,...
1709,2024-04-24T01:48:00-CME-001,2024-04-24T01:48Z,{'activityID': '2024-04-26T00:17:00-IPS-001'},2024-04-26T00:17:00-IPS-001
1710,2024-04-24T15:05:00-CME-001,2024-04-24T15:05Z,{'activityID': '2024-04-24T14:02:00-FLR-001'},2024-04-24T14:02:00-FLR-001
1711,2024-04-25T18:24:00-CME-001,2024-04-25T18:24Z,{'activityID': '2024-04-25T17:03:00-FLR-001'},2024-04-25T17:03:00-FLR-001
1712,2024-04-30T00:36:00-CME-001,2024-04-30T00:36Z,{'activityID': '2024-04-30T00:46:00-FLR-001'},2024-04-30T00:46:00-FLR-001


In [16]:
# Convert the 'GST_ActivityID' column to string format 
expanded_df['GST_ActivityID'] = expanded_df['GST_ActivityID'].astype(str)
# Convert startTime to datetime format  
expanded_df['startTime'] = pd.to_datetime(expanded_df['startTime'])
# Rename startTime to startTime_CME and activityID to cmeID
expanded_df.rename(columns={'startTime': 'startTime_CME'}, inplace=True)
expanded_df.rename(columns={'activityID': 'cmeID'}, inplace=True)
# Drop linkedEvents
expanded_df.drop(columns=['linkedEvents'], inplace=True)
# Verify that all steps were executed correctly
expanded_df

,cmeID,startTime_CME,GST_ActivityID
0,2013-05-01T03:12:00-CME-001,2013-05-01 03:12:00+00:00,2013-05-04T04:52:00-IPS-001
1,2013-05-03T22:36:00-CME-001,2013-05-03 22:36:00+00:00,2013-05-07T04:37:00-IPS-001
2,2013-05-09T19:29:00-CME-001,2013-05-09 19:29:00+00:00,2013-05-12T23:30:00-IPS-001
3,2013-05-13T02:54:00-CME-001,2013-05-13 02:54:00+00:00,2013-05-13T01:53:00-FLR-001
4,2013-05-13T02:54:00-CME-001,2013-05-13 02:54:00+00:00,2013-05-13T04:12:00-SEP-001
...,...,...,...
1709,2024-04-24T01:48:00-CME-001,2024-04-24 01:48:00+00:00,2024-04-26T00:17:00-IPS-001
1710,2024-04-24T15:05:00-CME-001,2024-04-24 15:05:00+00:00,2024-04-24T14:02:00-FLR-001
1711,2024-04-25T18:24:00-CME-001,2024-04-25 18:24:00+00:00,2024-04-25T17:03:00-FLR-001
1712,2024-04-30T00:36:00-CME-001,2024-04-30 00:36:00+00:00,2024-04-30T00:46:00-FLR-001


In [17]:
expanded_df

,cmeID,startTime_CME,GST_ActivityID
0,2013-05-01T03:12:00-CME-001,2013-05-01 03:12:00+00:00,2013-05-04T04:52:00-IPS-001
1,2013-05-03T22:36:00-CME-001,2013-05-03 22:36:00+00:00,2013-05-07T04:37:00-IPS-001
2,2013-05-09T19:29:00-CME-001,2013-05-09 19:29:00+00:00,2013-05-12T23:30:00-IPS-001
3,2013-05-13T02:54:00-CME-001,2013-05-13 02:54:00+00:00,2013-05-13T01:53:00-FLR-001
4,2013-05-13T02:54:00-CME-001,2013-05-13 02:54:00+00:00,2013-05-13T04:12:00-SEP-001
...,...,...,...
1709,2024-04-24T01:48:00-CME-001,2024-04-24 01:48:00+00:00,2024-04-26T00:17:00-IPS-001
1710,2024-04-24T15:05:00-CME-001,2024-04-24 15:05:00+00:00,2024-04-24T14:02:00-FLR-001
1711,2024-04-25T18:24:00-CME-001,2024-04-25 18:24:00+00:00,2024-04-25T17:03:00-FLR-001
1712,2024-04-30T00:36:00-CME-001,2024-04-30 00:36:00+00:00,2024-04-30T00:46:00-FLR-001


In [18]:
# We are only interested in CMEs related to GSTs so keep only rows where the GST_ActivityID column contains 'GST'
# use the method 'contains()' from the str library.
expanded_df = expanded_df[expanded_df['GST_ActivityID'].str.contains('GST')].reset_index(drop=True)
expanded_df


,cmeID,startTime_CME,GST_ActivityID
0,2013-06-02T20:24:00-CME-001,2013-06-02 20:24:00+00:00,2013-06-07T03:00:00-GST-001
1,2013-09-29T22:40:00-CME-001,2013-09-29 22:40:00+00:00,2013-10-02T03:00:00-GST-001
2,2013-12-04T23:12:00-CME-001,2013-12-04 23:12:00+00:00,2013-12-08T00:00:00-GST-001
3,2014-02-16T14:15:00-CME-001,2014-02-16 14:15:00+00:00,2014-02-19T03:00:00-GST-001
4,2014-02-18T01:25:00-CME-001,2014-02-18 01:25:00+00:00,2014-02-20T03:00:00-GST-001
...,...,...,...
56,2023-12-14T17:38:00-CME-001,2023-12-14 17:38:00+00:00,2023-12-18T06:00:00-GST-001
57,2024-02-28T17:48:00-CME-001,2024-02-28 17:48:00+00:00,2024-03-03T18:00:00-GST-001
58,2024-03-23T01:25:00-CME-001,2024-03-23 01:25:00+00:00,2024-03-24T12:00:00-GST-001
59,2024-03-23T01:48:00-CME-001,2024-03-23 01:48:00+00:00,2024-03-24T12:00:00-GST-001


### GST Data

In [19]:
# Set the base URL to NASA's DONKI API:
base_url = "https://api.nasa.gov/DONKI/"

# Set the specifier for Geomagnetic Storms (GST):
GST = "GST"

# Search for GSTs between a begin and end date
startDate = "2013-05-01"
endDate   = "2024-05-01"

# Build URL for GST
query_url_GST = f"{base_url}{GST}?startDate={startDate}&endDate={endDate}&api_key={NASA_API_KEY}"
print(f"GST Qury URL: {query_url_GST}")

GST Qury URL: https://api.nasa.gov/DONKI/GST?startDate=2013-05-01&endDate=2024-05-01&api_key=erI3TXXxMWNxoAuxWTN120s4vc27cAPLtvgkX2iF


In [20]:
# Make a "GET" request for the GST URL and store it in a variable named gst_response
gst_response = requests.get(query_url_GST)
if gst_response.status_code == 200:
    print("GST request successful!")
else:
    print(f"GST request not successful. Status code: {gst_response.status_code}")

GST request successful!


In [21]:
# Convert the response variable to json and store it as a variable named gst_json
gst_json = gst_response.json()
# Preview the first result in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(gst_json[0], indent=4))

{
    "gstID": "2013-06-01T01:00:00-GST-001",
    "startTime": "2013-06-01T01:00Z",
    "allKpIndex": [
        {
            "observedTime": "2013-06-01T01:00Z",
            "kpIndex": 6.0,
            "source": "NOAA"
        }
    ],
    "link": "https://webtools.ccmc.gsfc.nasa.gov/DONKI/view/GST/326/-1",
    "linkedEvents": [
        {
            "activityID": "2013-05-31T15:45:00-HSS-001"
        }
    ],
    "submissionTime": "2013-07-15T19:26Z",
    "versionId": 1
}


In [22]:
# Convert gst_json to a Pandas DataFrame  
gst_df = pd.DataFrame(gst_json)
# Keep only the columns: activityID, startTime, linkedEvents #activityID is not in this dataFrame.  Suggesting it should be the gstID instead.  Possible error?
gst_df = gst_df[['gstID', 'startTime', 'linkedEvents']]
gst_df.head()

,gstID,startTime,linkedEvents
0,2013-06-01T01:00:00-GST-001,2013-06-01T01:00Z,[{'activityID': '2013-05-31T15:45:00-HSS-001'}]
1,2013-06-07T03:00:00-GST-001,2013-06-07T03:00Z,[{'activityID': '2013-06-02T20:24:00-CME-001'}]
2,2013-06-29T03:00:00-GST-001,2013-06-29T03:00Z,None
3,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,[{'activityID': '2013-09-29T22:40:00-CME-001'}...
4,2013-12-08T00:00:00-GST-001,2013-12-08T00:00Z,[{'activityID': '2013-12-04T23:12:00-CME-001'}...


In [23]:
# Notice that the linkedEvents column allows us to identify the corresponding CME
# Remove rows with missing 'linkedEvents' since we won't be able to assign these to CME
gst_df = gst_df[gst_df['linkedEvents'].notna()].reset_index(drop=True)
gst_df.head()

,gstID,startTime,linkedEvents
0,2013-06-01T01:00:00-GST-001,2013-06-01T01:00Z,[{'activityID': '2013-05-31T15:45:00-HSS-001'}]
1,2013-06-07T03:00:00-GST-001,2013-06-07T03:00Z,[{'activityID': '2013-06-02T20:24:00-CME-001'}]
2,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,[{'activityID': '2013-09-29T22:40:00-CME-001'}...
3,2013-12-08T00:00:00-GST-001,2013-12-08T00:00Z,[{'activityID': '2013-12-04T23:12:00-CME-001'}...
4,2014-02-19T03:00:00-GST-001,2014-02-19T03:00Z,[{'activityID': '2014-02-16T14:15:00-CME-001'}...


In [24]:
# Notice that the linkedEvents sometimes contains multiple events per row
# Use the explode method to ensure that each row is one element. Ensure to reset the index and drop missing values.
gst_df = gst_df.explode('linkedEvents').reset_index(drop=True)

In [25]:
# Apply the extract_activityID_from_dict function to each row in the 'linkedEvents' column (you can use apply() and a lambda function)
# and create a new column called 'CME_ActivityID' using loc indexer:
def extract_activityID_from_dict(input_dict):
    try:
        return input_dict['activityID']
    except (ValueError,TypeError, KeyError):
        return None
gst_df.loc[:, 'CME_ActivityID'] = gst_df.loc[:, 'linkedEvents'].apply(extract_activityID_from_dict)
# Remove rows with missing CME_ActivityID, since we can't assign them to CMEs:
gst_df = gst_df[gst_df['CME_ActivityID'].notna()].reset_index(drop=True)
gst_df

,gstID,startTime,linkedEvents,CME_ActivityID
0,2013-06-01T01:00:00-GST-001,2013-06-01T01:00Z,{'activityID': '2013-05-31T15:45:00-HSS-001'},2013-05-31T15:45:00-HSS-001
1,2013-06-07T03:00:00-GST-001,2013-06-07T03:00Z,{'activityID': '2013-06-02T20:24:00-CME-001'},2013-06-02T20:24:00-CME-001
2,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,{'activityID': '2013-09-29T22:40:00-CME-001'},2013-09-29T22:40:00-CME-001
3,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,{'activityID': '2013-10-02T01:54:00-IPS-001'},2013-10-02T01:54:00-IPS-001
4,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,{'activityID': '2013-10-02T02:47:00-MPC-001'},2013-10-02T02:47:00-MPC-001
...,...,...,...,...
200,2024-03-24T12:00:00-GST-001,2024-03-24T12:00Z,{'activityID': '2024-03-23T01:48:00-CME-001'},2024-03-23T01:48:00-CME-001
201,2024-03-24T12:00:00-GST-001,2024-03-24T12:00Z,{'activityID': '2024-03-24T14:10:00-IPS-001'},2024-03-24T14:10:00-IPS-001
202,2024-03-24T12:00:00-GST-001,2024-03-24T12:00Z,{'activityID': '2024-03-24T16:25:00-MPC-001'},2024-03-24T16:25:00-MPC-001
203,2024-04-19T18:00:00-GST-001,2024-04-19T18:00Z,{'activityID': '2024-04-15T06:48:00-CME-001'},2024-04-15T06:48:00-CME-001


In [26]:
# Convert the 'CME_ActivityID' column to string format 
gst_df['CME_ActivityID'] = gst_df['CME_ActivityID'].astype(str)
# Convert the 'gstID' column to string format 
gst_df['gstID'] = gst_df['gstID'].astype(str)
# Convert startTime to datetime format  
gst_df['startTime'] = pd.to_datetime(gst_df['startTime'])
# Rename startTime to startTime_GST 
gst_df.rename(columns={'startTime': 'startTime_GST'}, inplace=True)
# Drop linkedEvents
if 'linkedEvents' in gst_df.columns:
    gst_df.drop(columns=['linkedEvents'], inplace=True)
# Verify that all steps were executed correctly
gst_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   gstID           205 non-null    object             
 1   startTime_GST   205 non-null    datetime64[ns, UTC]
 2   CME_ActivityID  205 non-null    object             
dtypes: datetime64[ns, UTC](1), object(2)
memory usage: 4.9+ KB


In [27]:
# We are only interested in GSTs related to CMEs so keep only rows where the CME_ActivityID column contains 'CME'
# use the method 'contains()' from the str library.  
gst_df = gst_df[gst_df['CME_ActivityID'].str.contains('CME')].reset_index(drop=True)
all_cme = gst_df['CME_ActivityID'].str.contains('CME').all() #GPT provided code for checking output
print(all_cme)
gst_df

True


,gstID,startTime_GST,CME_ActivityID
0,2013-06-07T03:00:00-GST-001,2013-06-07 03:00:00+00:00,2013-06-02T20:24:00-CME-001
1,2013-10-02T03:00:00-GST-001,2013-10-02 03:00:00+00:00,2013-09-29T22:40:00-CME-001
2,2013-12-08T00:00:00-GST-001,2013-12-08 00:00:00+00:00,2013-12-04T23:12:00-CME-001
3,2014-02-19T03:00:00-GST-001,2014-02-19 03:00:00+00:00,2014-02-16T14:15:00-CME-001
4,2014-02-20T03:00:00-GST-001,2014-02-20 03:00:00+00:00,2014-02-18T01:25:00-CME-001
...,...,...,...
56,2023-12-18T06:00:00-GST-001,2023-12-18 06:00:00+00:00,2023-12-14T17:38:00-CME-001
57,2024-03-03T18:00:00-GST-001,2024-03-03 18:00:00+00:00,2024-02-28T17:48:00-CME-001
58,2024-03-24T12:00:00-GST-001,2024-03-24 12:00:00+00:00,2024-03-23T01:25:00-CME-001
59,2024-03-24T12:00:00-GST-001,2024-03-24 12:00:00+00:00,2024-03-23T01:48:00-CME-001


### Merge both datatsets

In [28]:
# Now merge both datasets using 'gstID' and 'CME_ActivityID' for gst and 'GST_ActivityID' and 'cmeID' for cme. Use the 'left_on' and 'right_on' specifiers.
merged_df = pd.merge(
    gst_df,
    expanded_df,
    left_on=['gstID', 'CME_ActivityID'],
    right_on=['GST_ActivityID', 'cmeID'],
    how='inner'
)

In [29]:
# Verify that the new DataFrame has the same number of rows as cme and gst
print(merged_df.shape)
print(gst_df.shape)
print(expanded_df.shape)



(61, 6)
(61, 3)
(61, 3)


### Computing the time it takes for a CME to cause a GST

In [30]:
# Compute the time diff between startTime_GST and startTime_CME by creating a new column called `timeDiff`.
merged_df['timeDiff'] = merged_df['startTime_GST'] - merged_df['startTime_CME']
merged_df['timeDiff_days'] = merged_df['timeDiff'].dt.days


In [31]:
# Use describe() to compute the mean and median time 
# that it takes for a CME to cause a GST. 
print(merged_df['timeDiff_days'].describe())
merged_df.describe()

count    61.000000
mean      2.508197
std       1.074290
min       1.000000
25%       2.000000
50%       2.000000
75%       3.000000
max       6.000000
Name: timeDiff_days, dtype: float64


,timeDiff,timeDiff_days
count,61,61.000000
mean,2 days 23:29:26.557377049,2.508197
std,0 days 23:53:09.336914240,1.074290
min,1 days 08:36:00,1.000000
25%,2 days 04:00:00,2.000000
50%,2 days 19:51:00,2.000000
75%,3 days 13:35:00,3.000000
max,6 days 03:00:00,6.000000


### Exporting data in csv format

In [2]:
# Export data to CSV without the index
output_path = '/output/CME_GST_data.csv'
merged_df.to_csv(output_path, index=False)

NameError: name 'merged_df' is not defined